#Setup

In [1]:
import requests
import pandas as pd
import json
import plotly.express as px

In [2]:
def pull_data():
    crabs = []
    #as of apr 1 2022 7pm
    for id in range(1,21641):
        crabs.append(requests.get(f'https://api.crabada.com/public/crabada/info/{id}').json()['result'])
    with open('merged_json.json', 'w') as f:
        json.dump(crabs, f)

#Exploratory

In [3]:
crab_df = pd.concat([pd.read_json('1_10000.json'), pd.read_json('10000_21640.json').iloc[1: , :]]).reset_index(drop=True)
sales_df = pd.read_csv('2022-01-24-crabada-sales.csv')

In [4]:
sales_merged_df = sales_df.merge(crab_df, left_on='crab_id', right_on='id')

In [5]:
stats = ['hp','speed','damage','critical','armor']
normalized_stats_w = pd.concat([crab_df.id, (crab_df[stats]-crab_df[stats].min())/(crab_df[stats].max()-crab_df[stats].min())], axis=1)

In [6]:
# normalize stats and combine to get most efficient crabs
normalized_stats_l = pd.melt(normalized_stats_w, 'id',['hp','speed','damage','critical','armor'])
# px.bar(normalized_stats, 'id', 'value', color = 'variable')
normalized_stats_w['total'] = normalized_stats_w[['hp','speed','damage','critical','armor']].sum(axis=1)
px.histogram(normalized_stats_w.total)

In [7]:
print(crab_df.class_name.value_counts(),'\n')
print(crab_df.type.value_counts())

BULK       10106
PRIME       2433
GEM         1902
CRABOID     1691
SURGE       1542
RUINED      1422
SUNKEN      1328
ORGANIC     1201
Name: class_name, dtype: int64 

NORMAL     21561
GENESIS       64
Name: type, dtype: int64


In [8]:
px.histogram(crab_df[crab_df.price == crab_df.price].price)

In [9]:
# parts types
pd.concat([pd.DataFrame(crab_df[crab_df.type=='GENESIS'][i].value_counts().reset_index()) for i in ['shell_name','horn_name','mouth_name','eyes_name','pincers_name']], axis=1)
pd.concat([pd.DataFrame(crab_df[crab_df.type!='GENESIS'][i].value_counts().reset_index()) for i in ['shell_name','horn_name','mouth_name','eyes_name','pincers_name']], axis=1)

,index,shell_name,index,horn_name,index,mouth_name,index,eyes_name,index,pincers_name
0,Crava,2145,Rocco,2192,Crazurite,2258,Crava,1856,Crazurite,2143
1,Charoite,1871,Cropion,2115,Crava,1314,Crazurite,1847,Cropion,2004
2,Cropion,1535,Crazurite,1371,Charoite,1273,Rocco,1601,Charoite,1719
3,Chief,1423,Charoite,1322,C-Rex,1270,Cropion,1365,Cragma,1202
4,Rocco,1416,Cragma,1193,Cragma,1256,Charoite,1189,Chief,1186
...,...,...,...,...,...,...,...,...,...,...
59,Staro,143,Celon,136,Cralmon,145,Cragon,144,Crazor,149
60,Cralmon,141,Natura,135,Celon,145,Eva,144,Crawberry,145
61,Adam,140,Crobster,131,Cranana,142,Pearlio,143,Eva,142
62,Emeraldo,133,Staro,127,Adam,141,Twinner,132,Twinner,139


In [10]:
print(crab_df.breed_count.value_counts())
print(crab_df.breeding_fee.value_counts())

3    7096
2    5314
1    5291
0    3003
4     875
5      60
6       1
Name: breed_count, dtype: int64
{'CRA': 52.5, 'TUS': 25500}    7095
{'CRA': 52.5, 'TUS': 12750}    5312
{'CRA': 52.5, 'TUS': 8500}     5291
{'CRA': 52.5, 'TUS': 4250}     2947
{'CRA': 52.5, 'TUS': 38250}     874
{'CRA': 0, 'TUS': 0}             64
{'CRA': 52.5, 'TUS': None}       57
Name: breeding_fee, dtype: int64


# Dashboard
- We aim to understand what are the most popular types of crabs, stats, and other useful information we may come across. 
  - Most popular crabs, parts, types
  - Most popular stat distribution
  - Pricing
  - 

In [11]:
class_list = ['BULK','PRIME','GEM','CRABOID','RUINED','SUNKEN','SURGE','ORGANIC']
class_bool = {crabada_class: sales_merged_df.class_name == crabada_class for crabada_class in class_list}
class_bool['ALL'] = sales_merged_df.class_name != ''

def create_price_hist(class_name):
    return px.histogram(sales_merged_df[class_bool[class_name]].total_price, title='Sales Breakdown').update_layout(showlegend=False)

In [12]:
def create_sales_scatt(class_name):
    return px.scatter(sales_merged_df[class_bool[class_name]], y='total_price', x='sale_timestamp', title='Price Over Time').update_layout(showlegend=False) 

In [13]:
def create_quant_hist(class_name):
    return px.histogram(sales_merged_df[class_bool[class_name]].sale_timestamp, title='Sales Count Over Time').update_layout(showlegend=False) 

In [14]:
sales_merged_df['Date'] = pd.to_datetime(sales_merged_df['sale_timestamp']) - pd.to_timedelta(7, unit='d')
def create_tot_sales_line(class_name): 
  return px.line(sales_merged_df[class_bool[class_name]].groupby(pd.Grouper(key='Date', freq='W-MON'))['total_price'].sum(), title='Total Weekly Sales').update_layout(showlegend=False) 

In [15]:
def create_parts_tbl(part):
  return pd.DataFrame(sales_merged_df[f'{part}_name'].value_counts().reset_index()).set_axis([f'{part} name',f'{part} count'], axis=1)

In [16]:
parts = ['shell','horn','mouth','eyes','pincers']

def create_part_pareto(i):
  part_pareto = create_parts_tbl(i)
  part_pareto['percent'] = ((part_pareto[f'{i} count']/part_pareto[f'{i} count'].sum())*100).cumsum()
  par = make_subplots(specs=[[{"secondary_y": True}]])
  par.add_trace(go.Scatter(y=part_pareto.percent, x=part_pareto[f'{i} name']),secondary_y=True)
  par.add_trace(go.Bar(y=part_pareto[f'{i} count'], x=part_pareto[f'{i} name']))
  par.update_layout(margin={'t': 20})
  return par

In [17]:
class_bar = px.bar(pd.DataFrame(sales_merged_df.class_name.value_counts()).rename(columns={'class_name':'count'}),title='Sold Class Breakdown').update_layout(showlegend=False) 

In [18]:
# Most traded crabs
top_id_tbl = sales_merged_df.id.value_counts()

In [19]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def helper_viz(fig,stats):
    fig.data[0].visible = True
    fig.data[1].visible = True
    steps = []
    for i in range(len(stats)):
        step = dict(
            method="update",
            args=[{"visible": [False] * len(fig.data)},
                {"title": f"Created vs Sold: {stats[i]}"}], 
        )
        step["args"][0]["visible"][i*2] = True
        step["args"][0]["visible"][i*2+1] = True
        steps.append(step)

    fig.update_layout(sliders=[dict(active=0, steps=steps, currentvalue={'visible':False})])
    return fig

fig = go.Figure()
for i in stats:
  fig.add_trace(go.Histogram(x=crab_df[i].dropna(), name="All"))
  fig.add_trace(go.Histogram(x=sales_merged_df[i].dropna(), name='Sold'))
  fig.update_layout(barmode='overlay')
  fig.update_traces(opacity=0.75)

stats_hist = helper_viz(fig, stats)

In [20]:
def crab_stat(id):
  fig = px.line_polar(normalized_stats_w[normalized_stats_w.id == id][stats].melt(),
                      theta='variable',
                      r='value',
                      line_close=True,
                      title=f'Crabada stats: Crabada {id}')
  fig.update_traces(fill='toself')
  return fig

## Dashboard Setup

In [24]:
# !pip install jupyter-dash --upgrade
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_table

## Dashboard

In [25]:
app = JupyterDash(__name__)
Intro = """"
# Crabada Market Guide v.1.0
A basic guide to the crabada economy 

* Data extracted: Jan 24, 2022
"""

app.layout = html.Div([
    html.Div(
          dcc.Markdown(Intro),
    ),
    html.H2(id="Sales and Prices"),
    html.Div(
          dcc.Dropdown(id='class_dropdown',
                       options=[{'label': i, 'value': i} for i in class_list]+[{'label': 'ALL', 'value': 'ALL'}],
                       value='ALL')
    ),
    html.Div(
          dcc.Graph(id='price_hist'),
          style={'width': '49%', 'display': 'inline-block'}
    ),
    html.Div(
          dcc.Graph(id='sales_scatt'),
          style={'width': '49%', 'display': 'inline-block'}
    ),
    html.Div(
          dcc.Graph(id='quant_hist'),
          style={'width': '49%', 'display': 'inline-block'}
    ),
    html.Div(
          dcc.Graph(id='tot_sales_line'),
          style={'width': '49%', 'display': 'inline-block'}
    ),
    html.H2("Popular Classes and Parts"),
    html.Div(
          dcc.Graph(figure=class_bar),
          style={'width': '49%', 'display': 'inline-block', 'verticalAlign': 'top'}
    ),
    html.Div([
          dcc.Dropdown(id='part_dropdown', options=[
                                          {'label':f'Parts Breakdown: {i}', 'value':i} for i in parts
                                      ],
                                      value='shell',
                                  ),
          dcc.Graph(id='part_pareto')
          # dash_table.DataTable(id='part_table',
          #                      columns=[{"name": i, "id": i} for i in ['Part Name','Crab Count']],
          #                      editable=True,
          #                      fixed_rows={'headers': True},
          #                      style_table={'height': 320}),

    ],
    style={'width': '49%', 'display': 'inline-block', 'verticalAlign': 'top'}),
    html.H2("Most Sought Stats and Crabada Stat Checker"),
    html.Div(
          dcc.Graph(figure=stats_hist),
          style={'width': '49%', 'display': 'inline-block','verticalAlign': 'top'},
    ),
    html.Div([
              dcc.Input(id='crab_id', value='1', type='number'),
              dcc.Graph(id='stat_radar')
    ],
    style={'width': '49%', 'display': 'inline-block','verticalAlign': 'top'},
    )
])
    
@app.callback(
    Output("Sales and Prices","children"),
    Output("price_hist", "figure"),
    Output("sales_scatt","figure"),
    Output("quant_hist","figure"),
    Output("tot_sales_line","figure"),
    Input('class_dropdown', 'value')
    )

def update_tables(value):
    text = f"Sales and Prices: {value}"
    return text, create_price_hist(value), create_sales_scatt(value), create_quant_hist(value),create_tot_sales_line(value)

# @app.callback(
#     Output("part_table", "data"),
#     Output("part_table","columns"),
#     Input('part_dropdown', 'value')
#     )
# def update_table(value):
#     tbl = create_parts_tbl(value)
#     return tbl.to_dict('records'), [{"name": i, "id": i} for i in tbl.columns]

@app.callback(
    Output("part_pareto", "figure"),
    Input('part_dropdown', 'value')
    )
def update_graph(value):
    return create_part_pareto(value)

@app.callback(
    Output("stat_radar", "figure"),
    Input("crab_id","value")
)
def update_radar(value):
  return crab_stat(value)

app.run_server()

Dash app running on http://127.0.0.1:8050/


# Recommendations
- 3 actionable insights from the provided data with the intention of advising our investments team regarding market movements
  - unique users per month, check user hash then see trend
  - recommend ideal stats, class
  - ABC pareto on the parts

In [27]:
sales_merged_df['sale_timestamp'] = pd.to_datetime(sales_merged_df['sale_timestamp'])
px.line(sales_merged_df.groupby(pd.Grouper(key='Date', freq='W-MON'))['buyer'].nunique()/sales_merged_df.groupby(pd.Grouper(key='Date', freq='W-MON'))['seller'].nunique(), title='Crabada buyer to seller ratio')


In [28]:
#buyer+seller
px.line(sales_merged_df.groupby(pd.Grouper(key='Date', freq='W-MON'))['buyer'].nunique()+sales_merged_df.groupby(pd.Grouper(key='Date', freq='W-MON'))['seller'].nunique())

In [29]:
sales_merged_df.crabada_id.describe()

count    19685.000000
mean      8015.258065
std       4922.545833
min         17.000000
25%       4086.000000
50%       7069.000000
75%      11564.000000
max      21640.000000
Name: crabada_id, dtype: float64

In [30]:
#crab creation rate
px.histogram(crab_df.created_at)